## Problem Set #[7]
MACS 30150, Dr. Evans

Linghui Wu

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.metrics import classification_report, mean_squared_error

import warnings
warnings.filterwarnings("ignore")

### Question 1

#### (a)

In [2]:
drink = pd.read_csv("data/strongdrink.txt")
drink.head()

,cultivar,alco,malic,ash,alk,magn,tot_phen,flav,nonfl_phen,proanth,color_int,hue,OD280rat,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
X = drink[["alco", "malic", "tot_phen", "color_int"]]
y = drink["cultivar"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=20)
clf = LogisticRegression(random_state=0, solver="lbfgs", multi_class="multinomial").fit(X_train, y_train)

results = {"Variables": ["beta0", "beta1", "beta2", "beta3", "beta4"],
          "j=1": np.hstack((clf.intercept_[0], clf.coef_[0])),
          "j=2": np.hstack((clf.intercept_[1], clf.coef_[1]))}
pd.DataFrame(results)

,Variables,j=1,j=2
0,beta0,-24.027621,22.780736
1,beta1,1.701735,-1.466297
2,beta2,-0.265788,-0.332953
3,beta3,1.224101,0.663556
4,beta4,0.022506,-0.922682


In [4]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.87      1.00      0.93        13
           2       1.00      0.90      0.95        21
           3       1.00      1.00      1.00        10

    accuracy                           0.95        44
   macro avg       0.96      0.97      0.96        44
weighted avg       0.96      0.95      0.96        44



In [5]:
drink["cultivar"].value_counts()

2    71
1    59
3    46
Name: cultivar, dtype: int64

The error rates on the test set are **13%**, **0%**, and **0%** for group 1, 2 and 3 respectively.

The model is best at predicting **the third category** of cultivar because it has the highest f1-score among the three, but the most accurately predicted category(Group 3) is not the one with the most observations (Group 2).

In [6]:
mse = mean_squared_error(y_test, y_pred)
print("The MSE from the test set is {}.".format(mse))

The MSE from the test set is 0.045454545454545456.


#### (b)  LOOCV

In [7]:
Xvars = drink[["alco", "malic", "tot_phen", "color_int"]].values
yvars = drink["cultivar"]

N_loo = Xvars.shape[0]
loo = LeaveOneOut()
loo.get_n_splits(Xvars)

MSE_vec = np.zeros(N_loo)
y_test_arr = np.zeros(N_loo)
y_pred_arr = np.zeros(N_loo)
for train_index, test_index in loo.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    y_test_arr[test_index] = y_test
    
    LogReg = LogisticRegression(max_iter=200)
    LogReg.fit(X_train, y_train)
    
    y_pred = LogReg.predict(X_test)
    y_pred_arr[test_index] = y_pred
    if y_pred == y_test_arr[test_index]:
        MSE_vec[test_index] = 0
    else:
        MSE_vec[test_index] = 1
    
    print('MSE for test set', test_index, ' is', MSE_vec[test_index])

MSE for test set [0]  is [0.]
MSE for test set [1]  is [0.]
MSE for test set [2]  is [0.]
MSE for test set [3]  is [0.]
MSE for test set [4]  is [0.]
MSE for test set [5]  is [0.]
MSE for test set [6]  is [0.]
MSE for test set [7]  is [0.]
MSE for test set [8]  is [0.]
MSE for test set [9]  is [0.]
MSE for test set [10]  is [0.]
MSE for test set [11]  is [1.]
MSE for test set [12]  is [0.]
MSE for test set [13]  is [0.]
MSE for test set [14]  is [0.]
MSE for test set [15]  is [0.]
MSE for test set [16]  is [0.]
MSE for test set [17]  is [0.]
MSE for test set [18]  is [0.]
MSE for test set [19]  is [0.]
MSE for test set [20]  is [0.]
MSE for test set [21]  is [1.]
MSE for test set [22]  is [1.]
MSE for test set [23]  is [1.]
MSE for test set [24]  is [1.]
MSE for test set [25]  is [1.]
MSE for test set [26]  is [0.]
MSE for test set [27]  is [1.]
MSE for test set [28]  is [0.]
MSE for test set [29]  is [0.]
MSE for test set [30]  is [0.]
MSE for test set [31]  is [0.]
MSE for test set [

In [8]:
MSE_loo_mean = MSE_vec.mean()
MSE_loo_std = MSE_vec.std()
print("The LOOCV estimate MSE is {}.".format(MSE_loo_mean))
print("The LOOCV estimate standard error is {}.".format(MSE_loo_std))

The LOOCV estimate MSE is 0.13636363636363635.
The LOOCV estimate standard error is 0.3431742925123068.


In [9]:
print(classification_report(y_test_arr, y_pred_arr))

              precision    recall  f1-score   support

         1.0       0.84      0.78      0.81        59
         2.0       0.83      0.89      0.86        71
         3.0       0.96      0.93      0.95        46

    accuracy                           0.86       176
   macro avg       0.87      0.87      0.87       176
weighted avg       0.86      0.86      0.86       176



The error rates on the test set are **16%**, **17%**, and **4%** for group 1, 2 and 3 respectively.

#### (c)

In [10]:
k = 4
kf = KFold(n_splits=k, random_state=10, shuffle=True)
MSE_vec_kf = np.zeros(k)
kf.get_n_splits(Xvars)

y_test_arr = np.zeros(N_loo)
y_pred_arr = np.zeros(N_loo)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    
    LogReg = LogisticRegression(max_iter=300)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    
    y_test_arr[test_index] = y_test
    y_pred_arr[test_index] = y_pred
    
    MSE_vec_kf[k_ind] = (y_test != y_pred).mean()
    print("The K-fold estimate MSE for the test set {} is {}.".format(k_ind, MSE_vec_kf[k_ind]))
    k_ind += 1

The K-fold estimate MSE for the test set 0 is 0.22727272727272727.
The K-fold estimate MSE for the test set 1 is 0.22727272727272727.
The K-fold estimate MSE for the test set 2 is 0.13636363636363635.
The K-fold estimate MSE for the test set 3 is 0.09090909090909091.


In [11]:
MSE_kf_mean = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print("The K-fold estimate MSE is {}.".format(MSE_kf_mean))
print("The K-fold estimate standard error is {}.".format(MSE_kf_std))

The K-fold estimate MSE is 0.17045454545454544.
The K-fold estimate standard error is 0.05904718662166627.


In [13]:
print(classification_report(y_test_arr, y_pred_arr))

              precision    recall  f1-score   support

         1.0       0.78      0.73      0.75        59
         2.0       0.79      0.85      0.82        71
         3.0       0.96      0.93      0.95        46

    accuracy                           0.83       176
   macro avg       0.84      0.84      0.84       176
weighted avg       0.83      0.83      0.83       176



The error rates on the test set are **22%**, **21%**, and **4%** for group 1, 2 and 3 respectively. 

Compared to (a) and (b), the error rates increase greatly, while the error rate for group (3) stays the same in a leave-one-out and k-fold validation.

### Question 2